<a href="https://colab.research.google.com/github/sovunia-hub/mathematical-modeling-of-applied-problems/blob/main/3_two-step_gradient_descent_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np

an_sol = lambda x: 3/4 * x

In [ ]:
def findAF(N):
  a = 0
  b = 1
  h = np.abs(b - a) / N
  x = np.arange(N) * h + h / 2 + a
  A = np.zeros((N, N))
  for i in range(N):
    for j in range(N):
      if i == j:
        A[i, j] = h * x[i] * x[j] + 1
      else:
        A[i, j] = h * x[i] * x[j]
  return A, x

In [ ]:
def findSimpleU(N, eps):
  A, f = findAF(N)
  I = np.eye(N)
  u = np.zeros(N)
  B = I - A
  f_norm = np.linalg.norm(f)
  kol = 0
  while True:
    kol += 1
    u_prev = u
    u = B @ u + f
    if np.linalg.norm(u - u_prev) / f_norm < eps:
      return u, kol

In [ ]:
def findGradU(N, eps):
  A , f = findAF(N)
  u = np.zeros(N)
  f_norm = np.linalg.norm(f)
  kol = 0
  while True:
    kol += 1
    r = A @ u - f
    Ar = A.conj().T @ r
    u_prev = u
    u = u_prev - (np.linalg.norm(Ar)**2 / np.linalg.norm(A @ Ar)**2) * Ar
    if np.linalg.norm(u - u_prev) / f_norm < eps:
      return u, kol

In [ ]:
def findDoubleGradU(N, eps):
  A, f = findAF(N)
  u = np.zeros(N)
  r = A @ u - f
  u_prev = u
  Ar = A.conj().T @ r
  u = u_prev - (np.linalg.norm(Ar)**2 / np.linalg.norm(A @ Ar)**2) * Ar
  f_norm = np.linalg.norm(f)
  kol = 1
  while True:
    kol += 1
    r_prev = r
    r = A @ u - f
    Ar = A.conj().T @ r
    u_prev_prev = u_prev
    u_prev = u
    Ar_norm_square = np.linalg.norm(Ar) ** 2
    alpha_gamma = np.linalg.solve(np.array([[np.linalg.norm(r - r_prev) ** 2, Ar_norm_square],
                                            [Ar_norm_square, np.linalg.norm(A @ Ar) ** 2]]),
                                  np.array([0, Ar_norm_square]))
    u = u_prev - alpha_gamma.item(0) * (u_prev - u_prev_prev) - alpha_gamma.item(1) * Ar
    if np.linalg.norm(u - u_prev) / f_norm < eps:
      return u, kol

In [ ]:
N = 2000
eps = 0.0000000000000001
#0.0000000000000001
a = 0
b = 1
h = np.abs(b - a) / N
x = np.arange(N) * h + h / 2 + a
solution = an_sol(x)

%time uSimple, kolSimple = findSimpleU(N, eps)
print(f'Метод простых итераций: {kolSimple}')
print(f'Ошибка: {sum(np.abs(solution - uSimple)) / sum(np.abs(solution))}\n')

%time uGrad, kolGrad = findGradU(N, eps)
print(f'Метод градиентного спуска: {kolGrad}')
print(f'Ошибка: {sum(np.abs(solution - uGrad)) / sum(np.abs(solution))}\n')

%time uDoubleGrad, kolDoubleGrad = findDoubleGradU(N, eps)
print(f'Двухшаговый метод градиентного спуска: {kolDoubleGrad}')
print(f'Ошибка: {sum(np.abs(solution - uDoubleGrad)) / sum(np.abs(solution))}\n')

CPU times: user 2.63 s, sys: 33.4 ms, total: 2.67 s
Wall time: 2.62 s
Метод простых итераций: 35
Ошибка: 1.5625000268050974e-08

CPU times: user 2.16 s, sys: 126 ms, total: 2.28 s
Wall time: 2.14 s
Метод градиентного спуска: 4
Ошибка: 1.5625000266328215e-08

CPU times: user 2.18 s, sys: 120 ms, total: 2.3 s
Wall time: 2.16 s
Двухшаговый метод градиентного спуска: 4
Ошибка: 1.5625000260419455e-08

